<a href="https://colab.research.google.com/github/SUDHARSSHINI/AUTO_BOM/blob/main/Input_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = '/content/PLC datasheet - Sheet1.csv'
data = pd.read_csv(file_path)

# Clean column names by stripping any leading or trailing spaces
data.columns = data.columns.str.strip()

# Data Cleaning
# Extract numeric values from "Input port" and "Output port" columns and handle any non-numeric text
data['Input port'] = data['Input port'].str.extract('(\d+)').astype(float)
data['Output port'] = data['Output port'].str.extract('(\d+)').astype(float)

# Encode the "model number" column as it's the only feature we are using for prediction
label_encoder = LabelEncoder()
data['model number'] = label_encoder.fit_transform(data['model number'])

# Define features and target variables
X = data[['model number']]
y = data[['Input port', 'Output port']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Multi-output model training for predicting both input and output ports
multi_target_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
multi_target_model.fit(X_train, y_train)

# Predictions
y_pred = multi_target_model.predict(X_test)

# Calculate MAE for both predictions
mae_input = mean_absolute_error(y_test['Input port'], y_pred[:, 0])
mae_output = mean_absolute_error(y_test['Output port'], y_pred[:, 1])

print("Mean Absolute Error for Input port prediction:", mae_input)
print("Mean Absolute Error for Output port prediction:", mae_output)

# Get model number from user input and predict input/output ports
user_model_number = input("Enter the model number to predict Input and Output ports: ")

# Check if the model number is in the original data
if user_model_number in label_encoder.classes_:
    # Encode the model number
    encoded_model_number = label_encoder.transform([user_model_number])[0]
    # Create a DataFrame to match the feature format
    user_input_df = pd.DataFrame({'model number': [encoded_model_number]})
    # Make predictions
    predicted_ports = multi_target_model.predict(user_input_df)

    print(f"Predicted Input port for model {user_model_number}: {predicted_ports[0][0]}")
    print(f"Predicted Output port for model {user_model_number}: {predicted_ports[0][1]}")
else:
    print(f"The model number '{user_model_number}' is not found in the dataset.")


Mean Absolute Error for Input port prediction: 1.2088888888888887
Mean Absolute Error for Output port prediction: 1.2088888888888887
Enter the model number to predict Input and Output ports: NX1P2-9024DT
Predicted Input port for model NX1P2-9024DT: 15.88
Predicted Output port for model NX1P2-9024DT: 11.08
